In [ ]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np

In [4]:
network = cv2.dnn.readNet("yolov3.cfg", "yolov3.weights")

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20240521::readNetFromDarknet'


In [5]:
# Get the output layer names
layer_names = network.getLayerNames()
try:
    output_layers = [layer_names[i[0] - 1] for i in network.getUnconnectedOutLayers()]
except IndexError:
    output_layers = [layer_names[i - 1] for i in network.getUnconnectedOutLayers()]


NameError: name 'network' is not defined

In [18]:
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [19]:
# Initialize camera
capture = cv2.VideoCapture(0)  # Use 'rtsp://<ip>:<port>/stream' for an IP camera

# Check if the camera opened successfully
if not capture.isOpened():
    print("Error: Could not open video stream or file.")


In [20]:
# Loop through the frames
while True:
    ret, frame = capture.read()
    if not ret:
        break
        
    height, width, channels = frame.shape

    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    network.setInput(blob)
    outs = network.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    people_count = 0
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            if label == "person":
                people_count += 1
                color = (0, 255, 0)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, color, 2)

    cv2.putText(frame, f'People Count: {people_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Image", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Press ESC to exit
        break


KeyboardInterrupt: 

In [22]:
import csv
from datetime import datetime

# Initialize a file to store the history (run this once)
with open('people_count_history.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Timestamp', 'People Count'])

# Inside the while loop after counting people
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open('people_count_history.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, people_count])


In [ ]:
from IPython.display import FileLink

# Create a link to download the file
FileLink(r'people_count_history.csv')



In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('people_count_history.csv')
df.head()


In [21]:
# Release the camera and close all windows
capture.release()
cv2.destroyAllWindows()
